# MAST30034 Project 1
## Preprocessing

Potential questions:
How big of an impact do tourists have on taxi revenue?

In [1]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Project 1 - Preprocessing")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .getOrCreate()
)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/27 20:10:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
sdf = spark.read.parquet('P1 Data/yellow_tripdata_2019-01 (1).parquet')
sdf.show(1, vertical=True, truncate=100)

-RECORD 0------------------------------------
 VendorID              | 1                   
 tpep_pickup_datetime  | 2019-01-01 11:46:40 
 tpep_dropoff_datetime | 2019-01-01 11:53:20 
 passenger_count       | 1.0                 
 trip_distance         | 1.5                 
 RatecodeID            | 1.0                 
 store_and_fwd_flag    | N                   
 PULocationID          | 151                 
 DOLocationID          | 239                 
 payment_type          | 1                   
 fare_amount           | 7.0                 
 extra                 | 0.5                 
 mta_tax               | 0.5                 
 tip_amount            | 1.65                
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 9.95                
 congestion_surcharge  | null                
 airport_fee           | null                
only showing top 1 row



### Dropped Columns:

VendorID is used to indicate the provider who indicated the record, this inforation is not nesscary to know for this project as it has no effect of the taxis themselves. 

RatecodeID is used to indicate the kind of rate used (to calculate payment) during the ride. This is unescary information due to other columns.

store_and_fwd_flag is used to flag if the vechile had a conection to the service during payment, this information is unnessacy.

In [3]:
sdf = sdf.drop("VendorID", "RatecodeID", "store_and_fwd_flag")

### Combined columns

improvement_surcharge, mta_tax, extra, tolls_amount, airport_fee and congestion_surcharge are all various surcharges applied to the total fee to the customer. These on their own dont hold alot of information but the total will be kept.

In [9]:
sdf = sdf.na.fill(value=0,subset=["congestion_surcharge", "airport_fee"])

In [10]:
from pyspark.sql.functions import *
sdf = sdf.withColumn('surchange_amount', col("improvement_surcharge") + col("mta_tax") + col("extra") + col("tolls_amount") + col("airport_fee") + col("congestion_surcharge"))

In [ ]:
sdf = sdf.drop("extra", "mta_tax", "improvement_surcharge", "congestion_surcharge", "tolls_amount", "airport_fee")

### Renaming records

In [13]:
sdf = sdf.withColumnRenamed("tpep_pickup_datetime", "pickup_time")
sdf = sdf.withColumnRenamed("tpep_dropoff_datetime", "dropoff_time")
sdf = sdf.withColumnRenamed("PULocationID", "pick_up_location")
sdf = sdf.withColumnRenamed("DOLocationID", "drop_off_location")
sdf

pickup_time,dropoff_time,passenger_count,trip_distance,pick_up_location,drop_off_location,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,Surchanges,surchange_amount
2019-01-01 11:46:40,2019-01-01 11:53:20,1.0,1.5,151,239,1,7.0,0.5,0.5,1.65,0.0,0.3,9.95,0.0,0,1.3,1.3
2019-01-01 11:59:47,2019-01-01 12:18:59,1.0,2.6,239,246,1,14.0,0.5,0.5,1.0,0.0,0.3,16.3,0.0,0,1.3,1.3
2018-12-22 00:48:30,2018-12-22 00:52:40,3.0,0.0,236,236,1,4.5,0.5,0.5,0.0,0.0,0.3,5.8,0.0,0,1.3,1.3
2018-11-29 02:52:25,2018-11-29 02:55:45,5.0,0.0,193,193,2,3.5,0.5,0.5,0.0,0.0,0.3,7.55,0.0,0,1.3,1.3
2018-11-29 02:56:57,2018-11-29 02:58:33,5.0,0.0,193,193,2,52.0,0.0,0.5,0.0,0.0,0.3,55.55,0.0,0,0.8,0.8
2018-11-29 03:25:49,2018-11-29 03:28:26,5.0,0.0,193,193,2,3.5,0.5,0.5,0.0,5.76,0.3,13.31,0.0,0,7.06,7.06
2018-11-29 03:29:37,2018-11-29 03:33:43,5.0,0.0,193,193,2,52.0,0.0,0.5,0.0,0.0,0.3,55.55,0.0,0,0.8,0.8
2019-01-01 11:21:28,2019-01-01 11:28:37,1.0,1.3,163,229,1,6.5,0.5,0.5,1.25,0.0,0.3,9.05,0.0,0,1.3,1.3
2019-01-01 11:32:01,2019-01-01 11:45:39,1.0,3.7,229,7,1,13.5,0.5,0.5,3.7,0.0,0.3,18.5,0.0,0,1.3,1.3
2019-01-01 11:57:32,2019-01-01 12:09:32,2.0,2.1,141,234,1,10.0,0.5,0.5,1.7,0.0,0.3,13.0,0.0,0,1.3,1.3


### Converting units

The trip_distance is set to miles (imperial), thus for ease of analysis, this will be set to kilometres (metric)

In [19]:
sdf = sdf.withColumn("trip_distance", round(col("trip_distance") * 1.60934, 4)) 

As there are time stamps of the pickup an dropoff times, it may be handy to look into aspects affecting the duration of the trip, thus we will add a column for total duration in minutes:

In [ ]:
from pyspark.sql.functions import *
df.withColumn('from_timestamp',to_timestamp(col('from_timestamp')))\
  .withColumn('end_timestamp', current_timestamp())\
  .withColumn('DiffInSeconds',unix_timestamp("end_timestamp") - unix_timestamp('from_timestamp'))